In [93]:
from plotly.figure_factory import create_dendrogram
import numpy as np
import pandas as pd
import sqlite3

In [94]:
conn = sqlite3.connect('../data_processing/data/nba_shots.db')
df = pd.read_sql('select player, avg_distance, avg_shotX, accuracy, top_quarter from player_profiles_by_year where year = 2018', conn)
df.head()

,player,avg_distance,avg_shotX,accuracy,top_quarter
0,Aaron Brooks,14.642857,22.911905,0.380952,4
1,Aaron Gordon,13.178495,22.318172,0.416129,1
2,Aaron Harrison,18.927536,22.669565,0.275362,2
3,Aaron Holiday,17.765766,26.841441,0.423423,4
4,Aaron Jackson,9.333333,18.433333,0.666667,4


In [95]:
X = df.drop(columns='player').values
y = df.player.values
fig = create_dendrogram(X[:100,], labels=y[:100], color_threshold=10)
fig.show()

In [96]:
y[:100]

array(['Aaron Brooks', 'Aaron Gordon', 'Aaron Harrison', 'Aaron Holiday',
       'Aaron Jackson', 'Abdel Nader', 'Al Horford', 'Al Jefferson',
       'Al-Farouq Aminu', 'Alan Williams', 'Alec Burks', 'Alec Peters',
       'Alex Caruso', 'Alex Len', 'Alex Poythress', 'Alfonzo McKinnie',
       'Alize Johnson', 'Allen Crabbe', 'Allonzo Trier',
       'Amile Jefferson', 'Amir Johnson', 'Andre Drummond',
       'Andre Iguodala', 'Andre Ingram', 'Andre Roberson',
       'Andrew Harrison', 'Andrew White', 'Andrew Wiggins',
       'Anfernee Simons', 'Ante Žižić', 'Anthony Brown', 'Anthony Davis',
       'Anthony Tolliver', 'Antonio Blakeney', 'Antonius Cleveland',
       'Aron Baynes', 'Arron Afflalo', 'Austin Rivers', 'Avery Bradley',
       'Bam Adebayo', 'Ben McLemore', 'Ben Simmons', 'Bismack Biyombo',
       'Blake Griffin', 'Boban Marjanović', 'Bobby Portis',
       'Bogdan Bogdanović', 'Bojan Bogdanović', 'Brad Wanamaker',
       'Bradley Beal', 'Brandan Wright', 'Brandon Goodwin',
   

In [101]:
fig2 = create_dendrogram(X, labels=y, color_threshold=10, orientation='left')

target_player = 'Chris Paul'
tick_idx = np.argwhere(fig2.layout.yaxis.ticktext == target_player)[0][0]
# tick_val = fig2.layout.xaxis.tickvals[tick_idx]

zoomed_y_axis = [fig2.layout.yaxis.tickvals[tick_idx - 7], fig2.layout.yaxis.tickvals[tick_idx + 8]]
zoomed_x_axis = [0, 15]

fig2.update_xaxes(range=zoomed_x_axis)#, rangeslider=dict(visible=True))
fig2.update_yaxes(range=zoomed_y_axis)
fig2.show()